# Mother Tongues of Canada, by Knowledge of the official languages
by Grace Cowderoy

## Abstract
Canada is a nation with two official languages: English and French. According to the 2021 Statistics Canada census, 98.2% of Canadian residents speak at least one of the official languages. However, only 74.5% of the population consider one of the official languages as their first language (L1), also called their mother tongue - the language first learnt in childhood. For the other 25.6% of people, their mother tongue is a non-official language. 

The dataset Table 98-10-0175-01  Mother tongue by knowledge of official languages: Canada, provinces and territories, census metropolitan areas and census agglomerations with parts provides data from the 2021 Statistics Canada census on the respondent’s mother tongue, age at the time of census, gender, knowledge of the two official languages, and their location within Canada. 

This study proposes to weigh the factors that influence the knowledge of each official language. Research questions include: Does living in a particular area influence a speaker of a non-official language towards English or French? Does any single mother tongue influence a speaker towards English or French? What influence does the age of the speaker have on knowledge of the official languages? 

Classification techniques will be used to identify groups that may be more likely to speak each of the official languages, both the official languages, and those more likely to speak neither official language. As the dataset is imbalanced due to more speakers of English than French, balancing techniques such as oversampling and undersampling will be used. 


## Introduction
In Canada, the term ‘bilingual’ refers to speakers of both official languages, as opposed to the more general term where a speaker knows two different languages. The distribution of these bilingual speakers is not uniform across Canada, and neither is the distribution of speakers of non-official languages. 

The population of Canada has a variety of both indigenous and non-indigenous languages as their mother tongue. Canada is home to 81 living indigenous languages (Ethnologue), with many immigrants bringing their own native language to Canada. 

There are approximately 7,000 languages currently in use in the world (Ethnologue). These languages can be structured into families. English belongs to the Germanic branch of the Indo-European family, while French belongs to the Romance branch of the same family. It has been established that generally it is easier to acquire a second language (L2) when it is closely related to the learner’s first language (L1) (Gampe, A 2021). As such, within Canada, it may reasonably be expected that those speakers with an L1 more closely related to French would have greater knowledge of French. Another factor may be the location of the individual within Canada - policies in different provinces may promote one official language over another. The age of the individual can affect their language skills, as it is generally easier to acquire a second language in childhood compared to adulthood. Further, language policy in Canada has changed over time and varies between the provinces and territories, which suggests that age may be a factor in knowledge of the official languages depending on the policies in place over time. 

This is an investigation of the dataset Statistics Canada. Table 98-10-0175-01  Mother tongue by knowledge of official languages: Canada, provinces and territories, census metropolitan areas and census agglomerations with parts, https://doi.org/10.25318/9810017501-eng 

The dataset was released 2022-08-17 and comes from the 2021 Census of Population of Canada. Due to the recent release of the dataset, it does not yet appear to have been cited on Web of Science (As of 2023-10-07)

The dataset has multiple dimensions, including age, gender, geographic location, mother tongue of respondent, respondent's knowledge of the official languages (French and English). These dimensions include aggregates as individual records, e.g. Canada as a record, Ontario, Toronto. Part of the data cleaning will involve converting these aggregations into their tree structures. 

Importing the required libraries, bringing in the dataset motherTongues Table 98-10-0175-01 and doing the initial exploratory data analysis. 

In [3]:
#Data Handling and Modeling Tools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ydata_profiling import ProfileReport
from ydata_profiling.model.typeset import ProfilingTypeSet
from bigtree import dataframe_to_tree_by_relation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, KFold,cross_val_score,cross_validate
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, brier_score_loss, matthews_corrcoef
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
import graphviz

/home/grace/anaconda3/lib/python3.11/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/grace/anaconda3/lib/python3.11/site-packages/visions/backends/shared/nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


In [4]:
#Benchmarking Tools
import pyperformance
import pyperf
import memory_profiler
from scipy.stats import ttest_ind

In [5]:
%load_ext memory_profiler

In [6]:
motherTongues_filepath = '/home/grace/Documents/CapstoneProject/MotherTongues/98100175.csv'
motherTongues_data = pd.read_csv(motherTongues_filepath)
motherTongues_data.describe()

,REF_DATE,Knowledge of official languages (5):Total - Knowledge of official languages[1],Symbol,Knowledge of official languages (5):English only[2],Symbol.1,Knowledge of official languages (5):French only[3],Symbol.2,Knowledge of official languages (5):English and French[4],Symbol.3,Knowledge of official languages (5):Neither English nor French[5],Symbol.4
count,2591730.0,2.591730e+06,0.0,2.591730e+06,0.0,2.591730e+06,0.0,2.591730e+06,0.0,2.591730e+06,0.0
mean,2021.0,1.067793e+03,NaN,7.499710e+02,NaN,1.010610e+02,NaN,1.881355e+02,NaN,2.855237e+01,NaN
std,0.0,6.512709e+04,NaN,4.737592e+04,NaN,1.002769e+04,NaN,1.314044e+04,NaN,1.603253e+03,NaN
min,2021.0,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN
25%,2021.0,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN
50%,2021.0,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN
75%,2021.0,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,NaN
max,2021.0,3.662096e+07,NaN,2.526166e+07,NaN,4.087895e+06,NaN,6.581680e+06,NaN,6.897250e+05,NaN


In [7]:
print(len(motherTongues_data))
list(motherTongues_data.columns)

2591730


['REF_DATE',
 'GEO',
 'DGUID',
 'Gender (3)',
 'Age (15A)',
 'Mother tongue (331)',
 'Coordinate',
 'Knowledge of official languages (5):Total - Knowledge of official languages[1]',
 'Symbol',
 'Knowledge of official languages (5):English only[2]',
 'Symbol.1',
 'Knowledge of official languages (5):French only[3]',
 'Symbol.2',
 'Knowledge of official languages (5):English and French[4]',
 'Symbol.3',
 'Knowledge of official languages (5):Neither English nor French[5]',
 'Symbol.4']

In [8]:
motherTongues_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2591730 entries, 0 to 2591729
Data columns (total 17 columns):
 #   Column                                                                          Dtype  
---  ------                                                                          -----  
 0   REF_DATE                                                                        int64  
 1   GEO                                                                             object 
 2   DGUID                                                                           object 
 3   Gender (3)                                                                      object 
 4   Age (15A)                                                                       object 
 5   Mother tongue (331)                                                             object 
 6   Coordinate                                                                      object 
 7   Knowledge of official languages (5):Total - K

In [9]:
genderList = motherTongues_data["Gender (3)"].unique()


In [10]:
profile = ProfileReport(motherTongues_data, title = "Profiling Report", explorative=True)
##profile

In [11]:
#profile.to_file('/home/grace/Documents/CapstoneProject/MotherTongues-Canada/EDA.html')


The YData Profiling identified a number of empty features and some uniform features. These will be handled in the next steps and YData Profile will be run again. 

In [12]:
motherTongues_data.isnull().sum()

REF_DATE                                                                                0
GEO                                                                                     0
DGUID                                                                                   0
Gender (3)                                                                              0
Age (15A)                                                                               0
Mother tongue (331)                                                                     0
Coordinate                                                                              0
Knowledge of official languages (5):Total - Knowledge of official languages[1]          0
Symbol                                                                            2591730
Knowledge of official languages (5):English only[2]                                     0
Symbol.1                                                                          2591730
Knowledge 

In [13]:
df = motherTongues_data[[c for c in motherTongues_data.columns if 'Symbol' not in c]]
list(df.columns)

['REF_DATE',
 'GEO',
 'DGUID',
 'Gender (3)',
 'Age (15A)',
 'Mother tongue (331)',
 'Coordinate',
 'Knowledge of official languages (5):Total - Knowledge of official languages[1]',
 'Knowledge of official languages (5):English only[2]',
 'Knowledge of official languages (5):French only[3]',
 'Knowledge of official languages (5):English and French[4]',
 'Knowledge of official languages (5):Neither English nor French[5]']

In [14]:
df=df.loc[:,df.columns != 'REF_DATE']
df=df.loc[:,df.columns != 'DGUID']

In [15]:
df.isnull().sum()

GEO                                                                               0
Gender (3)                                                                        0
Age (15A)                                                                         0
Mother tongue (331)                                                               0
Coordinate                                                                        0
Knowledge of official languages (5):Total - Knowledge of official languages[1]    0
Knowledge of official languages (5):English only[2]                               0
Knowledge of official languages (5):French only[3]                                0
Knowledge of official languages (5):English and French[4]                         0
Knowledge of official languages (5):Neither English nor French[5]                 0
dtype: int64

In [16]:
profileDF = ProfileReport(df, title = "Profiling Report (Reduced)", explorative=True)
#profileDF.to_file('/home/grace/Documents/CapstoneProject/MotherTongues-Canada/EDA-reduced.html')

In [17]:
list(df.columns)

['GEO',
 'Gender (3)',
 'Age (15A)',
 'Mother tongue (331)',
 'Coordinate',
 'Knowledge of official languages (5):Total - Knowledge of official languages[1]',
 'Knowledge of official languages (5):English only[2]',
 'Knowledge of official languages (5):French only[3]',
 'Knowledge of official languages (5):English and French[4]',
 'Knowledge of official languages (5):Neither English nor French[5]']

The dataset includes aggregations as individual rows. The features 'Knowledge of Official languages' for Single responses (row 1, coordinate 1.1.1.2) is the sum of row 2 and row 5 - Official languages and Non-official languages respectively. Part of the data cleaning will require separating out these aggregations. 

Upon reviewing the metadata - each Geographical area has a unique member ID and is listed with its parent Member ID. 
E.g. Canada has ID 1; Nova Scotia has ID 10 and parent ID 1; and Halifax has ID 12 and parent member 10 (Nova Scotia)

Similarly for age - Total Age has ID 1; 25 to 64 years has ID 9 and parent ID 1; 25 to 34 years has ID 10 and parent ID 9. 

Mother tongues, Gender, knowledge of official languages have similar encoding available in the metadata. 

The next sections will review the tree hierarchies for the independent variables and flatten them to reduce dimensionality. 

In [18]:
df.head(n=8)

,GEO,Gender (3),Age (15A),Mother tongue (331),Coordinate,Knowledge of official languages (5):Total - Knowledge of official languages[1],Knowledge of official languages (5):English only[2],Knowledge of official languages (5):French only[3],Knowledge of official languages (5):English and French[4],Knowledge of official languages (5):Neither English nor French[5]
0,Canada,Total - Gender,Total - Age,Total - Mother tongue,1.1.1.1,36620955,25261655,4087895,6581680,689725
1,Canada,Total - Gender,Total - Age,Single responses,1.1.1.2,35145265,24306165,4029960,6130560,678580
2,Canada,Total - Gender,Total - Age,Official languages,1.1.1.3,27296445,18325325,3734010,5226490,10620
3,Canada,Total - Gender,Total - Age,English,1.1.1.4,20107200,18285580,5990,1806605,9025
4,Canada,Total - Gender,Total - Age,French,1.1.1.5,7189245,39740,3728020,3419880,1595
5,Canada,Total - Gender,Total - Age,Non-official languages,1.1.1.6,7848820,5980845,295950,904065,667955
6,Canada,Total - Gender,Total - Age,Indigenous languages,1.1.1.7,148895,123580,10995,8785,5535
7,Canada,Total - Gender,Total - Age,Algonquian languages,1.1.1.8,97125,79020,10730,5625,1760


The following section brings in the metadata files as individual dataframes. The metadata provides the id of the element and the id of the parent element. Any element that is listed with an ID that is not in the Parent ID column thus is not an aggregation. Each data element is labelled as being a parent or not. This equates to True whenever it appears in the Parent ID list and False when it does not. This will allow us to consider those values that are not aggregations. 

In [19]:
motherTonguesMeta_filepath = '/home/grace/Documents/CapstoneProject/MotherTongues-Canada/MetaData_Geography.csv'
geoData = pd.read_csv(motherTonguesMeta_filepath)

#geoData.head()
geoReduced = geoData[['Member Name','Member ID','Parent Member ID']]

isParent = geoReduced['Member ID'].isin(geoReduced['Parent Member ID'])

geoReduced['Is Parent Loc'] = isParent

#geoReduced.tail()
censusMA = geoReduced.loc[geoReduced['Is Parent Loc']== False]

censusMA.head()

/tmp/ipykernel_34980/4052823195.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geoReduced['Is Parent Loc'] = isParent


,Member Name,Member ID,Parent Member ID,Is Parent Loc
2,"Corner Brook (CA), N.L.",3,2.0,False
3,"Gander (CA), N.L.",4,2.0,False
4,"Grand Falls-Windsor (CA), N.L.",5,2.0,False
5,"St. John's (CMA), N.L.",6,2.0,False
7,"Charlottetown (CA), P.E.I.",8,7.0,False


This following section shows the hierarchical structure of the GEO attribute - the geographical location within Canada. Note that CMA refers to Census Metropolitan Area while CA refers to Census Agglomeration. https://www12.statcan.gc.ca/census-recensement/2021/ref/dict/az/Definition-eng.cfm?ID=geo009 

In [20]:
geoTree = dataframe_to_tree_by_relation(geoReduced, 'Member ID','Parent Member ID')
geoTree.show(attr_list = ['Member Name'])


1 [Member Name=Canada]
├── 2 [Member Name=Newfoundland and Labrador]
│   ├── 3 [Member Name=Corner Brook (CA), N.L.]
│   ├── 4 [Member Name=Gander (CA), N.L.]
│   ├── 5 [Member Name=Grand Falls-Windsor (CA), N.L.]
│   └── 6 [Member Name=St. John's (CMA), N.L.]
├── 7 [Member Name=Prince Edward Island]
│   ├── 8 [Member Name=Charlottetown (CA), P.E.I.]
│   └── 9 [Member Name=Summerside (CA), P.E.I.]
├── 10 [Member Name=Nova Scotia]
│   ├── 11 [Member Name=Cape Breton (CA), N.S.]
│   ├── 12 [Member Name=Halifax (CMA), N.S.]
│   ├── 13 [Member Name=Kentville (CA), N.S.]
│   ├── 14 [Member Name=New Glasgow (CA), N.S.]
│   └── 15 [Member Name=Truro (CA), N.S.]
├── 16 [Member Name=New Brunswick]
│   ├── 17 [Member Name=Bathurst (CA), N.B.]
│   ├── 18 [Member Name=Campbellton (CA), N.B./Que.]
│   │   ├── 19 [Member Name=Campbellton (New Brunswick part) (CA), N.B.]
│   │   └── 20 [Member Name=Campbellton (Quebec part) (CA), Que.]
│   ├── 21 [Member Name=Edmundston (CA), N.B.]
│   ├── 22 [Member

In [21]:
motherTonguesMeta_filepath = '/home/grace/Documents/CapstoneProject/MotherTongues-Canada/MetaData_MotherTongue.csv'
motherTongueData = pd.read_csv(motherTonguesMeta_filepath)

motherTongueData.head()
motherTongueData = motherTongueData[['Member Name','Member ID','Parent Member ID']]

isLanguage = motherTongueData['Member ID'].isin(motherTongueData['Parent Member ID'])

motherTongueData['Is Parent Lang'] = isLanguage
motherTongueData.tail(8)

,Member Name,Member ID,Parent Member ID,Is Parent Lang
323,Hungarian,324,321.0,False
324,"Other languages, n.i.e.",325,98.0,False
325,Multiple responses,326,1.0,True
326,English and French,327,326.0,False
327,English and non-official language(s),328,326.0,False
328,French and non-official language(s),329,326.0,False
329,"English, French and non-official language(s)",330,326.0,False
330,Multiple non-official languages,331,326.0,False


In [22]:
motherTongueTree = dataframe_to_tree_by_relation(motherTongueData,'Member ID','Parent Member ID')
motherTongueTree.show(attr_list = ['Member Name'])

1 [Member Name=Total - Mother tongue]
├── 2 [Member Name=Single responses]
│   ├── 3 [Member Name=Official languages]
│   │   ├── 4 [Member Name=English]
│   │   └── 5 [Member Name=French]
│   └── 6 [Member Name=Non-official languages]
│       ├── 7 [Member Name=Indigenous languages]
│       │   ├── 8 [Member Name=Algonquian languages]
│       │   │   ├── 9 [Member Name=Blackfoot]
│       │   │   ├── 10 [Member Name=Cree-Innu languages]
│       │   │   │   ├── 11 [Member Name=Atikamekw]
│       │   │   │   ├── 12 [Member Name=Cree languages]
│       │   │   │   │   ├── 13 [Member Name=Ililimowin (Moose Cree)]
│       │   │   │   │   ├── 14 [Member Name=Inu Ayimun (Southern East Cree)]
│       │   │   │   │   ├── 15 [Member Name=Iyiyiw-Ayimiwin (Northern East Cree)]
│       │   │   │   │   ├── 16 [Member Name=Nehinawewin (Swampy Cree)]
│       │   │   │   │   ├── 17 [Member Name=Nehiyawewin (Plains Cree)]
│       │   │   │   │   ├── 18 [Member Name=Nihithawiwin (Woods Cree)]
│       │  

In [23]:
motherTongueLangs = motherTongueData.loc[(motherTongueData['Is Parent Lang']== False) & (motherTongueData['Parent Member ID'] != 326)]
motherTongueLangs.head(10)

,Member Name,Member ID,Parent Member ID,Is Parent Lang
3,English,4,3.0,False
4,French,5,3.0,False
8,Blackfoot,9,8.0,False
10,Atikamekw,11,10.0,False
12,Ililimowin (Moose Cree),13,12.0,False
13,Inu Ayimun (Southern East Cree),14,12.0,False
14,Iyiyiw-Ayimiwin (Northern East Cree),15,12.0,False
15,Nehinawewin (Swampy Cree),16,12.0,False
16,Nehiyawewin (Plains Cree),17,12.0,False
17,Nihithawiwin (Woods Cree),18,12.0,False


In [24]:
motherTonguesMeta_filepath = '/home/grace/Documents/CapstoneProject/MotherTongues-Canada/MetaData_Age.csv'
motherTongueAge = pd.read_csv(motherTonguesMeta_filepath)
motherTongueAge = motherTongueAge[['Member Name','Member ID','Parent Member ID']]

isAgeGroup = motherTongueAge['Member ID'].isin(motherTongueAge['Parent Member ID'])
motherTongueAge['Is Parent Age']=isAgeGroup
#motherTongueAge.head()

In [25]:
motherTonguesMeta_filepath = '/home/grace/Documents/CapstoneProject/MotherTongues-Canada/MetaData_Gender.csv'
motherTongueGender = pd.read_csv(motherTonguesMeta_filepath)
motherTongueGender = motherTongueGender[['Member Name','Member ID','Parent Member ID']]

isGenderGroup = motherTongueGender['Member ID'].isin(motherTongueGender['Parent Member ID'])
motherTongueGender['Is Parent Gender']=isGenderGroup
motherTongueGender.head()

,Member Name,Member ID,Parent Member ID,Is Parent Gender
0,Total - Gender,1,NaN,True
1,Men+,2,1.0,False
2,Women+,3,1.0,False


This next section takes the metadata dataframes from the previous section and joins them on the motherTongues data from earlier. This provides the option to filter out aggregations or keep only aggregations. 

In [26]:
df = df.join(motherTongueData.set_index('Member Name'), on='Mother tongue (331)', rsuffix='_lang')
df = df.join(geoReduced.set_index('Member Name'), on='GEO',rsuffix='_geo')
df = df.join(motherTongueAge.set_index('Member Name'), on='Age (15A)', rsuffix='_age')
df = df.join(motherTongueGender.set_index('Member Name'),on='Gender (3)', rsuffix='_gender')
list(df.columns)

['GEO',
 'Gender (3)',
 'Age (15A)',
 'Mother tongue (331)',
 'Coordinate',
 'Knowledge of official languages (5):Total - Knowledge of official languages[1]',
 'Knowledge of official languages (5):English only[2]',
 'Knowledge of official languages (5):French only[3]',
 'Knowledge of official languages (5):English and French[4]',
 'Knowledge of official languages (5):Neither English nor French[5]',
 'Member ID',
 'Parent Member ID',
 'Is Parent Lang',
 'Member ID_geo',
 'Parent Member ID_geo',
 'Is Parent Loc',
 'Member ID_age',
 'Parent Member ID_age',
 'Is Parent Age',
 'Member ID_gender',
 'Parent Member ID_gender',
 'Is Parent Gender']

Excluding aggregate values - those that have parent = true. Also excluding the multiple mother tongues for consistency. 

In [27]:

dfFlat = df.loc[(df['Is Parent Lang'] == False) & (df['Parent Member ID'] != 326) & (df['Is Parent Loc'] == False)& (df['Is Parent Age'] == False)& (df['Is Parent Gender'] == False)]



In [28]:
dfFlat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 911856 entries, 35420 to 2591723
Data columns (total 22 columns):
 #   Column                                                                          Non-Null Count   Dtype  
---  ------                                                                          --------------   -----  
 0   GEO                                                                             911856 non-null  object 
 1   Gender (3)                                                                      911856 non-null  object 
 2   Age (15A)                                                                       911856 non-null  object 
 3   Mother tongue (331)                                                             911856 non-null  object 
 4   Coordinate                                                                      911856 non-null  object 
 5   Knowledge of official languages (5):Total - Knowledge of official languages[1]  911856 non-null  int64  
 6  

In [29]:
dfFlatReduced = dfFlat[['GEO', 'Gender (3)', 'Age (15A)', 'Mother tongue (331)', 'Coordinate',  'Knowledge of official languages (5):English only[2]', 'Knowledge of official languages (5):French only[3]', 'Knowledge of official languages (5):English and French[4]', 'Knowledge of official languages (5):Neither English nor French[5]']]
dfFlatReduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 911856 entries, 35420 to 2591723
Data columns (total 9 columns):
 #   Column                                                             Non-Null Count   Dtype 
---  ------                                                             --------------   ----- 
 0   GEO                                                                911856 non-null  object
 1   Gender (3)                                                         911856 non-null  object
 2   Age (15A)                                                          911856 non-null  object
 3   Mother tongue (331)                                                911856 non-null  object
 4   Coordinate                                                         911856 non-null  object
 5   Knowledge of official languages (5):English only[2]                911856 non-null  int64 
 6   Knowledge of official languages (5):French only[3]                 911856 non-null  int64 
 7   Knowledge of off

In [30]:
dfFlatReduced['Mother tongue (331)'] = dfFlatReduced['Mother tongue (331)'].astype('category')
dfFlatReduced['GEO'] = dfFlatReduced['GEO'].astype('category')

/tmp/ipykernel_34980/1589484963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFlatReduced['Mother tongue (331)'] = dfFlatReduced['Mother tongue (331)'].astype('category')
/tmp/ipykernel_34980/1589484963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFlatReduced['GEO'] = dfFlatReduced['GEO'].astype('category')


Repeating the YData Profile Report again, this time with the cleaned and flattened data. 

In [31]:
profileDF = ProfileReport(dfFlatReduced, title = "Profiling Report (Reduced)", explorative=True)
#profileDF.to_file('/home/grace/Documents/CapstoneProject/MotherTongues-Canada/EDA-Flattened-reduced.html')

Now that the data has been cleaned and flattened, we are ready to start applying machine learning models. The first step is to list which features are the target variables . We will also need to encode the categorical features as numbers; fortunately this has already been done, as the Coordinate feature is a combination of the member IDs. The string can be expanded by the deliminator. We will also divide the data into training and test sets at this time. 

In [32]:
col_namesX = list(dfFlatReduced[[c for c in dfFlatReduced.columns if 'Knowledge' not in c]].columns)
col_namesY = list(dfFlatReduced[[c for c in dfFlatReduced.columns if 'Knowledge' in c]].columns)
#dfFlatReduced['Total People'] = dfFlatReduced[col_namesY].sum(axis='columns')
#dfFlatReduced['Not Empty'] = np.where(dfFlatReduced['Total People'] >0, True, False)

dfFlatReduced.head()
#print(col_namesX, col_namesY)
X_Categories = dfFlatReduced[col_namesX]
Y_Categories = dfFlatReduced[col_namesY]


In [33]:
dfFlatReduced[col_namesY].sum(axis='columns')

35420      525
35421        0
35425        0
35427        0
35429        0
          ... 
2591718      0
2591720      0
2591721      0
2591722      0
2591723      0
Length: 911856, dtype: int64

In [34]:

X_Categories[['Geography','Gender','Age','Mother Tongue']]=X_Categories['Coordinate'].str.split('.', expand=True)
X_Categories.head()

/tmp/ipykernel_34980/2786190882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_Categories[['Geography','Gender','Age','Mother Tongue']]=X_Categories['Coordinate'].str.split('.', expand=True)
/tmp/ipykernel_34980/2786190882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_Categories[['Geography','Gender','Age','Mother Tongue']]=X_Categories['Coordinate'].str.split('.', expand=True)
/tmp/ipykernel_34980/2786190882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

,GEO,Gender (3),Age (15A),Mother tongue (331),Coordinate,Geography,Gender,Age,Mother Tongue
35420,"Corner Brook (CA), N.L.",Men+,0 to 4 years,English,3.2.3.4,3,2,3,4
35421,"Corner Brook (CA), N.L.",Men+,0 to 4 years,French,3.2.3.5,3,2,3,5
35425,"Corner Brook (CA), N.L.",Men+,0 to 4 years,Blackfoot,3.2.3.9,3,2,3,9
35427,"Corner Brook (CA), N.L.",Men+,0 to 4 years,Atikamekw,3.2.3.11,3,2,3,11
35429,"Corner Brook (CA), N.L.",Men+,0 to 4 years,Ililimowin (Moose Cree),3.2.3.13,3,2,3,13


In [35]:
X = X_Categories[['Geography','Gender','Age','Mother Tongue']]
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 911856 entries, 35420 to 2591723
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Geography      911856 non-null  object
 1   Gender         911856 non-null  object
 2   Age            911856 non-null  object
 3   Mother Tongue  911856 non-null  object
dtypes: object(4)
memory usage: 34.8+ MB


In [36]:
#mlb = MultiLabelBinarizer(classes=Y_Categories.columns)
#y = mlb.fit(Y_Categories)
#y

In [37]:
Y_Categories.head()

,Knowledge of official languages (5):English only[2],Knowledge of official languages (5):French only[3],Knowledge of official languages (5):English and French[4],Knowledge of official languages (5):Neither English nor French[5]
35420,525,0,0,0
35421,0,0,0,0
35425,0,0,0,0
35427,0,0,0,0
35429,0,0,0,0


In [38]:
'''
Y_Categories['Target Vector'] = Y_Categories.loc[:, Y_Categories.columns != 'Target Vector'].values.tolist()
Y_Categories.head()
Y = Y_Categories['Target Vector']
ts = pd.Series(pd.arrays.SparseArray(Y_Categories['Target Vector']))
ts.head()
Y = MultiLabelBinarizer()
Y.fit(Y_Categories['Target Vector'])
'''

"\nY_Categories['Target Vector'] = Y_Categories.loc[:, Y_Categories.columns != 'Target Vector'].values.tolist()\nY_Categories.head()\nY = Y_Categories['Target Vector']\nts = pd.Series(pd.arrays.SparseArray(Y_Categories['Target Vector']))\nts.head()\nY = MultiLabelBinarizer()\nY.fit(Y_Categories['Target Vector'])\n"

Originally had the test set as 30% of the output data, however this crashed when the decision_tree predictor function was run. As such, reduced the test_size down to 25%. This still gives 227,964 records for the test set. 

## Model evaluation

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y_Categories, test_size=0.25,random_state=42)
print(len(X_train),len(Y_train),len(X_test),len(Y_test))

683892 683892 227964 227964


In [40]:
Y_EngOnly_train = Y_train['Knowledge of official languages (5):English only[2]']
Y_EngOnly_test = Y_test['Knowledge of official languages (5):English only[2]']
Y_FrOnly_train = Y_train['Knowledge of official languages (5):French only[3]']
Y_FrOnly_test = Y_test['Knowledge of official languages (5):French only[3]']
Y_Bilang_train = Y_train['Knowledge of official languages (5):English and French[4]']
Y_Bilang_test = Y_test['Knowledge of official languages (5):English and French[4]']
Y_Neither_train = Y_train['Knowledge of official languages (5):Neither English nor French[5]']
Y_Neither_test = Y_test['Knowledge of official languages (5):Neither English nor French[5]']

In [40]:
X_names = list(X_train.columns)
X_train.head()

,Geography,Gender,Age,Mother Tongue
768626,52,2,13,45
594718,40,3,12,243
2052890,138,3,8,29
1665636,112,3,8,45
549509,37,3,11,50


Model evaluation - time and memory usages

In [ ]:
%%timeit
mtTreeEngOnly = DecisionTreeClassifier(max_depth=8)
mtTreeEngOnly=mtTreeEngOnly.fit(X_train,Y_EngOnly_train)

In [42]:
%%memit
mtTreeEngOnly = DecisionTreeClassifier(max_depth=8)
mtTreeEngOnly=mtTreeEngOnly.fit(X_train,Y_EngOnly_train)


peak memory: 2153.30 MiB, increment: 0.12 MiB


Model Training Time: 

2.2 s ± 209 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

peak memory: 2792.07 MiB, increment: 0.62 MiB


In [43]:
%%timeit

crossValEngOnly = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_EngOnly_train),X_train,Y_EngOnly_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))


/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warni

12.3 s ± 608 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%memit
crossValEngOnly = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_EngOnly_train),X_train,Y_EngOnly_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))


/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


peak memory: 3384.18 MiB, increment: 1246.60 MiB


Cross Validation Time: 

15.2 s ± 1.67 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

peak memory: 3606.84 MiB, increment: 814.77 MiB

Each of the models will have k-fold cross-validation applied on the training data. We will start with k=5. 

In [41]:
#%%timeit

crossValEngOnly5 = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_EngOnly_train),X_train,Y_EngOnly_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))
crossValFrOnly5 = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_FrOnly_train),X_train,Y_FrOnly_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))
crossValBilang5 = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_Bilang_train),X_train,Y_Bilang_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))
crossValNeither5 = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_Neither_train),X_train,Y_Neither_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))


/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation time on all four models: 
51.3 s ± 1.91 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [42]:
print("5 Fold Cross Validation Decision Tree")
print("Metrics for English Only \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValEngOnly5['test_matthews_corrcoef'].mean(), crossValEngOnly5['test_matthews_corrcoef'].std(), crossValEngOnly5['test_neg_mean_absolute_error'].mean(),crossValEngOnly5['test_neg_mean_absolute_error'].std(), crossValEngOnly5['test_neg_mean_squared_error'].mean(), crossValEngOnly5['test_neg_mean_squared_error'].std()))

print("\nMatthews Correlation ", crossValEngOnly5['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValEngOnly5['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,crossValEngOnly5['test_neg_mean_squared_error'])
print("Fit time ", crossValEngOnly5['fit_time'].mean())
print("Score time ", crossValEngOnly5['score_time'].mean())
print("\n\n")
print("Metrics for French Only \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValFrOnly5['test_matthews_corrcoef'].mean(),crossValFrOnly5['test_matthews_corrcoef'].std(), crossValFrOnly5['test_neg_mean_absolute_error'].mean(),crossValFrOnly5['test_neg_mean_absolute_error'].std(), crossValFrOnly5['test_neg_mean_squared_error'].mean(),crossValFrOnly5['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValFrOnly5['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValFrOnly5['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValFrOnly5['test_neg_mean_squared_error'])
print("Fit time ", crossValFrOnly5['fit_time'].mean())
print("Score time ", crossValFrOnly5['score_time'].mean())
print("\n\n")
print("Metrics for Both English and French \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValBilang5['test_matthews_corrcoef'].mean(),crossValBilang5['test_matthews_corrcoef'].std(), crossValBilang5['test_neg_mean_absolute_error'].mean(),crossValBilang5['test_neg_mean_absolute_error'].std(), crossValBilang5['test_neg_mean_squared_error'].mean(), crossValBilang5['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValBilang5['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValBilang5['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValBilang5['test_neg_mean_squared_error'])
print("Fit time ", crossValBilang5['fit_time'].mean())
print("Score time ", crossValBilang5['score_time'].mean())
print("\n\n")
print("Metrics for Neither English nor French \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValNeither5['test_matthews_corrcoef'].mean(),crossValNeither5['test_matthews_corrcoef'].std(), crossValNeither5['test_neg_mean_absolute_error'].mean(),crossValNeither5['test_neg_mean_absolute_error'].std(), crossValNeither5['test_neg_mean_squared_error'].mean(),crossValNeither5['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValNeither5['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValNeither5['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValNeither5['test_neg_mean_squared_error'])
print("Fit time ", crossValNeither5['fit_time'].mean())
print("Score time ", crossValNeither5['score_time'].mean())

5 Fold Cross Validation Decision Tree
Metrics for English Only 

 0.12 mean and 0.01 std Matthews Correlation Coefficient 
 -21.83 mean and 2.12 std Mean Absolute Error 
 -858212.45 mean and 288894.46 std Mean Squared Error

Matthews Correlation  [0.13137006 0.12247655 0.12899182 0.12812986 0.11096891]
Mean Absolute Error  [-25.86720184 -21.81040218 -20.44005615 -19.90133647 -21.11582272]
Mean Squared Error  [-1402143.06472485  -760499.36960352  -535290.1479039   -794113.63102984
  -799016.02231353]
Fit time  1.6166035175323485
Score time  1.1869153022766112



Metrics for French Only 

 0.23 mean and 0.01 std Matthews Correlation Coefficient 
 -2.94 mean and 0.74 std Mean Absolute Error 
 -97864.67 mean and 44123.85 std Mean Squared Error

Matthews Correlation  [0.23536455 0.22094063 0.22023311 0.22613083 0.22255916]
Mean Absolute Error  [-4.17205858 -2.15120742 -3.31957625 -2.30965506 -2.75175832]
Mean Squared Error [-183711.62020486  -70907.46496173  -94854.32324643  -64099.87497989

In [43]:
#%%timeit
engOnlyLR = cross_validate(LinearRegression().fit(X_train,Y_EngOnly_train), X_train,Y_EngOnly_train,cv=5,scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
frOnlyLR = cross_validate(LinearRegression().fit(X_train,Y_FrOnly_train), X_train,Y_FrOnly_train,cv=5,scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
bilangLR = cross_validate(LinearRegression().fit(X_train,Y_Bilang_train), X_train,Y_Bilang_train,cv=5,scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
neitherLR = cross_validate(LinearRegression().fit(X_train,Y_Neither_train), X_train,Y_Neither_train,cv=5,scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))

In [45]:
print("5 Fold Cross Validation Linear Regression")
print("Metrics for English Only \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (engOnlyLR['test_neg_mean_absolute_error'].mean(),engOnlyLR['test_neg_mean_absolute_error'].std(), engOnlyLR['test_neg_mean_squared_error'].mean(), engOnlyLR['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", engOnlyLR['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,engOnlyLR['test_neg_mean_squared_error'])
print("Fit time ", engOnlyLR['fit_time'].mean())
print("Score time ", engOnlyLR['score_time'].mean())
print("\n\n")
print("5 Fold Cross Validation")
print("Metrics for French Only \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (frOnlyLR['test_neg_mean_absolute_error'].mean(),frOnlyLR['test_neg_mean_absolute_error'].std(), frOnlyLR['test_neg_mean_squared_error'].mean(), frOnlyLR['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", frOnlyLR['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,frOnlyLR['test_neg_mean_squared_error'])
print("Fit time ", frOnlyLR['fit_time'].mean())
print("Score time ", frOnlyLR['score_time'].mean())
print("\n\n")

print("Metrics for Bilingual \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (bilangLR['test_neg_mean_absolute_error'].mean(),bilangLR['test_neg_mean_absolute_error'].std(), bilangLR['test_neg_mean_squared_error'].mean(), bilangLR['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", bilangLR['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,bilangLR['test_neg_mean_squared_error'])
print("Fit time ", bilangLR['fit_time'].mean())
print("Score time ", bilangLR['score_time'].mean())
print("\n\n")
print("Metrics for Neither \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (neitherLR['test_neg_mean_absolute_error'].mean(),neitherLR['test_neg_mean_absolute_error'].std(), neitherLR['test_neg_mean_squared_error'].mean(), neitherLR['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", neitherLR['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,neitherLR['test_neg_mean_squared_error'])
print("Fit time ", neitherLR['fit_time'].mean())
print("Score time ", neitherLR['score_time'].mean())

5 Fold Cross Validation Linear Regression
Metrics for English Only 

  -52.59 mean and 0.99 std Mean Absolute Error 
 -880639.42 mean and 252979.02 std Mean Squared Error
Mean Absolute Error  [-53.07174397 -51.17338107 -54.16888675 -52.23153414 -52.30040055]
Mean Squared Error  [ -672839.64412356  -642753.8863441  -1318410.16388243 -1004068.67105767
  -765124.71042919]
Fit time  0.49303603172302246
Score time  0.09295215606689453



5 Fold Cross Validation
Metrics for French Only 

  -9.15 mean and 0.43 std Mean Absolute Error 
 -103347.65 mean and 51837.02 std Mean Squared Error
Mean Absolute Error  [-9.87889024 -9.3232363  -9.04385056 -8.89659613 -8.62868648]
Mean Squared Error  [-168978.88100548 -106003.27285378 -149003.59848437  -62882.65592441
  -29869.8365656 ]
Fit time  0.46958150863647463
Score time  0.09231619834899903



5 Fold Cross Validation
Metrics for Bilingual 

  -15.03 mean and 0.84 std Mean Absolute Error 
 -182587.78 mean and 147774.86 std Mean Squared Error
Mean Ab

In [51]:
engOnlyRF5 = cross_validate(RandomForestRegressor().fit(X_train,Y_EngOnly_train),X_train,Y_EngOnly_train,cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
frOnlyRF5 = cross_validate(RandomForestRegressor().fit(X_train,Y_FrOnly_train),X_train,Y_FrOnly_train,cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
bilangRF5 = cross_validate(RandomForestRegressor().fit(X_train,Y_Bilang_train),X_train,Y_Bilang_train,cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
neitherRF5 = cross_validate(RandomForestRegressor().fit(X_train,Y_Neither_train),X_train,Y_Neither_train,cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))

In [53]:
print("5 Fold Cross Validation Random Forest")
print("Metrics for English Only \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (engOnlyRF5['test_neg_mean_absolute_error'].mean(),engOnlyRF5['test_neg_mean_absolute_error'].std(), engOnlyRF5['test_neg_mean_squared_error'].mean(), engOnlyRF5['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", engOnlyRF5['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,engOnlyRF5['test_neg_mean_squared_error'])
print("Fit time ", engOnlyRF5['fit_time'].mean())
print("Score time ", engOnlyRF5['score_time'].mean())
print("\n\n")
print("Metrics for French Only \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (frOnlyRF5['test_neg_mean_absolute_error'].mean(),frOnlyRF5['test_neg_mean_absolute_error'].std(), frOnlyRF5['test_neg_mean_squared_error'].mean(), frOnlyRF5['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", frOnlyRF5['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,frOnlyRF5['test_neg_mean_squared_error'])
print("Fit time ", frOnlyRF5['fit_time'].mean())
print("Score time ", frOnlyRF5['score_time'].mean())
print("\n\n")

print("Metrics for Bilingual \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (bilangRF5['test_neg_mean_absolute_error'].mean(),bilangRF5['test_neg_mean_absolute_error'].std(), bilangRF5['test_neg_mean_squared_error'].mean(), bilangRF5['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", bilangRF5['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,bilangRF5['test_neg_mean_squared_error'])
print("Fit time ", bilangRF5['fit_time'].mean())
print("Score time ", bilangRF5['score_time'].mean())
print("\n\n")
print("Metrics for Neither \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (neitherRF5['test_neg_mean_absolute_error'].mean(),neitherRF5['test_neg_mean_absolute_error'].std(), neitherRF5['test_neg_mean_squared_error'].mean(), neitherRF5['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", neitherRF5['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,neitherRF5['test_neg_mean_squared_error'])
print("Fit time ", neitherRF5['fit_time'].mean())
print("Score time ", neitherRF5['score_time'].mean())

5 Fold Cross Validation Random Forest
Metrics for English Only 

  -4.54 mean and 0.36 std Mean Absolute Error 
 -19735.79 mean and 10160.64 std Mean Squared Error
Mean Absolute Error  [-5.14129362 -4.23863276 -4.78010974 -4.20623273 -4.33219999]
Mean Squared Error  [-18984.87132111 -15663.40343413 -39427.0434452  -12922.63288818
 -11680.98944361]
Fit time  100.28504242897034
Score time  2.2673812389373778



Metrics for French Only 

  -0.77 mean and 0.16 std Mean Absolute Error 
 -5675.54 mean and 3519.88 std Mean Squared Error
Mean Absolute Error  [-0.9349626  -0.82231593 -0.77170269 -0.83421749 -0.46421318]
Mean Squared Error  [ -4829.12237864  -3838.14972337  -8257.377451   -10785.38419402
   -667.64762369]
Fit time  68.8757480621338
Score time  0.9868324279785157



Metrics for Bilingual 

  -1.60 mean and 0.40 std Mean Absolute Error 
 -7427.72 mean and 5664.99 std Mean Squared Error
Mean Absolute Error  [-2.32594002 -1.62424751 -1.5274463  -1.3793015  -1.12920024]
Mean Squared 

To evaluate whether there is a difference between the number of folds, the above models will be run again with k=10

In [46]:

crossValEngOnly = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_EngOnly_train),X_train,Y_EngOnly_train, cv=10, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))
crossValFrOnly = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_FrOnly_train),X_train,Y_FrOnly_train, cv=10, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))
crossValBilang = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_Bilang_train),X_train,Y_Bilang_train, cv=10, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))
crossValNeither = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_Neither_train),X_train,Y_Neither_train, cv=10, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))


/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [55]:
print("10 Fold Cross Validation Decision Tree")
print("Metrics for English Only \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" % (crossValEngOnly['test_matthews_corrcoef'].mean(), crossValEngOnly['test_matthews_corrcoef'].std(), crossValEngOnly['test_neg_mean_absolute_error'].mean(),crossValEngOnly['test_neg_mean_absolute_error'].std(), crossValEngOnly['test_neg_mean_squared_error'].mean(), crossValEngOnly['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValEngOnly['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValEngOnly['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValEngOnly['test_neg_mean_squared_error'])
print("Fit time ", crossValEngOnly['fit_time'].mean())
print("Score time ", crossValEngOnly['score_time'].mean())
print("\n\n")
print("Metrics for French Only \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValFrOnly['test_matthews_corrcoef'].mean(),crossValFrOnly['test_matthews_corrcoef'].std(), crossValFrOnly['test_neg_mean_absolute_error'].mean(),crossValFrOnly['test_neg_mean_absolute_error'].std(), crossValFrOnly['test_neg_mean_squared_error'].mean(),crossValFrOnly['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValFrOnly['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValFrOnly['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValFrOnly['test_neg_mean_squared_error'])
print("Fit time ", crossValFrOnly['fit_time'].mean())
print("Score time ", crossValFrOnly['score_time'].mean())
print("\n\n")
print("Metrics for Both English and French \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValBilang['test_matthews_corrcoef'].mean(),crossValBilang['test_matthews_corrcoef'].std(), crossValBilang['test_neg_mean_absolute_error'].mean(),crossValBilang['test_neg_mean_absolute_error'].std(), crossValBilang['test_neg_mean_squared_error'].mean(), crossValBilang['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValBilang['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValBilang['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValBilang['test_neg_mean_squared_error'])
print("Fit time ", crossValBilang['fit_time'].mean())
print("Score time ", crossValBilang['score_time'].mean())
print("\n\n")
print("Metrics for Neither English nor French \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValNeither['test_matthews_corrcoef'].mean(),crossValNeither['test_matthews_corrcoef'].std(), crossValNeither['test_neg_mean_absolute_error'].mean(),crossValNeither['test_neg_mean_absolute_error'].std(), crossValNeither['test_neg_mean_squared_error'].mean(),crossValNeither['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValNeither['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValNeither['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValNeither['test_neg_mean_squared_error'])
print("Fit time ", crossValNeither['fit_time'].mean())
print("Score time ", crossValNeither['score_time'].mean())

10 Fold Cross Validation Decision Tree
Metrics for English Only 

 0.13 mean and 0.01 std Matthews Correlation Coefficient 
 -21.69 mean and 3.80 std Mean Absolute Error 
 -858493.21 mean and 546313.55 std Mean Squared Error

Matthews Correlation  [0.12691609 0.12856789 0.1245371  0.13030507 0.12926085 0.10759225
 0.133247   0.12638975 0.13453907 0.13113882]
Mean Absolute Error  [-29.70931423 -18.14066384 -23.03294389 -16.79180862 -23.43256957
 -21.81198731 -25.11302987 -17.77413034 -22.62015821 -18.44616824]
Mean Squared Error [-2109391.78388653  -333486.51703465  -682417.41800582  -323084.25806782
 -1181825.72562839  -694815.48933308 -1188222.34021553  -387899.22319379
 -1268787.46801386  -415001.88151603]
Fit time  1.791214895248413
Score time  0.26606791019439696



Metrics for French Only 

 0.23 mean and 0.01 std Matthews Correlation Coefficient 
 -2.90 mean and 1.04 std Mean Absolute Error 
 -97869.21 mean and 76416.25 std Mean Squared Error

Matthews Correlation  [0.22917734 0.

In [54]:
engOnlyLR10 = cross_validate(LinearRegression().fit(X_train,Y_EngOnly_train), X_train,Y_EngOnly_train,cv=10,scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
frOnlyLR10 = cross_validate(LinearRegression().fit(X_train,Y_FrOnly_train), X_train,Y_FrOnly_train,cv=10,scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
bilangLR10 = cross_validate(LinearRegression().fit(X_train,Y_Bilang_train), X_train,Y_Bilang_train,cv=10,scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
neitherLR10 = cross_validate(LinearRegression().fit(X_train,Y_Neither_train), X_train,Y_Neither_train,cv=10,scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))

In [56]:
print("10 Fold Cross Validation Linear Regression")
print("Metrics for English Only \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (engOnlyLR10['test_neg_mean_absolute_error'].mean(),engOnlyLR10['test_neg_mean_absolute_error'].std(), engOnlyLR10['test_neg_mean_squared_error'].mean(), engOnlyLR10['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", engOnlyLR10['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,engOnlyLR10['test_neg_mean_squared_error'])
print("Fit time ", engOnlyLR10['fit_time'].mean())
print("Score time ", engOnlyLR10['score_time'].mean())
print("\n\n")
print("Metrics for French Only \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (frOnlyLR10['test_neg_mean_absolute_error'].mean(),frOnlyLR10['test_neg_mean_absolute_error'].std(), frOnlyLR10['test_neg_mean_squared_error'].mean(), frOnlyLR10['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", frOnlyLR10['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,frOnlyLR10['test_neg_mean_squared_error'])
print("Fit time ", frOnlyLR10['fit_time'].mean())
print("Score time ", frOnlyLR10['score_time'].mean())
print("\n\n")

print("Metrics for Bilingual \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (bilangLR10['test_neg_mean_absolute_error'].mean(),bilangLR10['test_neg_mean_absolute_error'].std(), bilangLR10['test_neg_mean_squared_error'].mean(), bilangLR10['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", bilangLR10['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,bilangLR10['test_neg_mean_squared_error'])
print("Fit time ", bilangLR10['fit_time'].mean())
print("Score time ", bilangLR10['score_time'].mean())
print("\n\n")
print("Metrics for Neither \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (neitherLR10['test_neg_mean_absolute_error'].mean(),neitherLR10['test_neg_mean_absolute_error'].std(), neitherLR10['test_neg_mean_squared_error'].mean(), neitherLR10['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", neitherLR10['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,neitherLR10['test_neg_mean_squared_error'])
print("Fit time ", neitherLR10['fit_time'].mean())
print("Score time ", neitherLR10['score_time'].mean())

10 Fold Cross Validation Linear Regression
Metrics for English Only 

  -52.59 mean and 2.30 std Mean Absolute Error 
 -880644.96 mean and 466775.15 std Mean Squared Error
Mean Absolute Error  [-53.86773035 -52.20682107 -53.50353326 -48.13117946 -54.12662407
 -55.2559924  -49.42187826 -55.24522957 -53.3118241  -50.80630384]
Mean Squared Error  [ -936262.78304307  -409407.02524254  -984251.55332466  -301297.47414691
  -857543.13576213 -1779261.86816357  -415251.15498046 -1592922.68164257
  -920990.95000658  -609261.0206065 ]
Fit time  0.5085257053375244
Score time  0.0502605676651001



Metrics for French Only 

  -9.15 mean and 0.83 std Mean Absolute Error 
 -103346.88 mean and 79672.01 std Mean Squared Error
Mean Absolute Error  [-10.22134811 -10.09792638  -9.99098213  -8.83644225  -8.12520487
  -9.98425067  -9.35313536  -8.09433565  -8.22377253  -8.554991  ]
Mean Squared Error  [-182008.28185291 -155940.12632477 -154174.87666177  -57834.51341612
  -31825.98058817 -266183.3575285  -10

In [57]:
engOnlyRF10 = cross_validate(RandomForestRegressor().fit(X_train,Y_EngOnly_train),X_train,Y_EngOnly_train,cv=10, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
frOnlyRF10 = cross_validate(RandomForestRegressor().fit(X_train,Y_FrOnly_train),X_train,Y_FrOnly_train,cv=10, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
bilangRF10 = cross_validate(RandomForestRegressor().fit(X_train,Y_Bilang_train),X_train,Y_Bilang_train,cv=10, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))
neitherRF10 = cross_validate(RandomForestRegressor().fit(X_train,Y_Neither_train),X_train,Y_Neither_train,cv=10, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error'))

In [58]:
print("10 Fold Cross Validation Random Forest")
print("Metrics for English Only \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (engOnlyRF10['test_neg_mean_absolute_error'].mean(),engOnlyRF10['test_neg_mean_absolute_error'].std(), engOnlyRF10['test_neg_mean_squared_error'].mean(), engOnlyRF10['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", engOnlyRF10['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,engOnlyRF10['test_neg_mean_squared_error'])
print("Fit time ", engOnlyRF10['fit_time'].mean())
print("Score time ", engOnlyRF10['score_time'].mean())
print("\n\n")
print("Metrics for French Only \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (frOnlyRF10['test_neg_mean_absolute_error'].mean(),frOnlyRF10['test_neg_mean_absolute_error'].std(), frOnlyRF10['test_neg_mean_squared_error'].mean(), frOnlyRF10['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", frOnlyRF10['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,frOnlyRF10['test_neg_mean_squared_error'])
print("Fit time ", frOnlyRF10['fit_time'].mean())
print("Score time ", frOnlyRF10['score_time'].mean())
print("\n\n")

print("Metrics for Bilingual \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (bilangRF10['test_neg_mean_absolute_error'].mean(),bilangRF10['test_neg_mean_absolute_error'].std(), bilangRF10['test_neg_mean_squared_error'].mean(), bilangRF10['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", bilangRF10['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,bilangRF10['test_neg_mean_squared_error'])
print("Fit time ", bilangRF10['fit_time'].mean())
print("Score time ", bilangRF10['score_time'].mean())
print("\n\n")
print("Metrics for Neither \n\n  %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (neitherRF10['test_neg_mean_absolute_error'].mean(),neitherRF10['test_neg_mean_absolute_error'].std(), neitherRF10['test_neg_mean_squared_error'].mean(), neitherRF10['test_neg_mean_squared_error'].std()))
print("Mean Absolute Error ", neitherRF10['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,neitherRF10['test_neg_mean_squared_error'])
print("Fit time ", neitherRF10['fit_time'].mean())
print("Score time ", neitherRF10['score_time'].mean())

10 Fold Cross Validation Random Forest
Metrics for English Only 

  -4.03 mean and 0.40 std Mean Absolute Error 
 -18281.01 mean and 10088.35 std Mean Squared Error
Mean Absolute Error  [-4.48220135 -4.53961983 -4.19477621 -3.44032885 -4.22680621 -4.02042288
 -3.45028586 -4.21496001 -4.21380924 -3.49722397]
Mean Squared Error  [-21436.14324788 -12284.16869937 -21575.55006342  -9450.32922177
 -36490.07267422 -36172.07502318  -7090.76508846 -15913.12681345
 -10884.95496198 -11512.95300403]
Fit time  114.55119349956513
Score time  1.2340061664581299



Metrics for French Only 

  -0.73 mean and 0.28 std Mean Absolute Error 
 -5438.14 mean and 6289.41 std Mean Squared Error
Mean Absolute Error  [-0.84105571 -0.69924404 -1.08033894 -0.66462808 -0.49262016 -0.90634971
 -1.28014666 -0.38377151 -0.45536344 -0.45407302]
Mean Squared Error  [ -5017.21388229  -2679.04220412  -7625.26197503  -2616.28563793
  -1491.43922751 -11851.62814192 -21210.43553346   -286.5903574
   -688.42556073   -915.0977

In [49]:
'''
class_values = dfFlatReduced[col_namesY].unique()
feature_names = list(X_train.columns())
dot_data = export_graphviz(mtTree, out_file=None, feature_names=)


dot_data = export_graphviz(mtTree, 
                           out_file=None, 
                           filled=True, 
                           special_characters= True, 
                           class_names=col_namesY,
                           feature_names=X_names)
graph = graphviz.Source(dot_data, format="png")
graph
'''

'\nclass_values = dfFlatReduced[col_namesY].unique()\nfeature_names = list(X_train.columns())\ndot_data = export_graphviz(mtTree, out_file=None, feature_names=)\n\n\ndot_data = export_graphviz(mtTree, \n                           out_file=None, \n                           filled=True, \n                           special_characters= True, \n                           class_names=col_namesY,\n                           feature_names=X_names)\ngraph = graphviz.Source(dot_data, format="png")\ngraph\n'

In [50]:
'''
decision_pred_EngOnly = mtTreeEngOnly.predict(X_test)
#decision_pred_EngOnly.shape
mean_absolute_error(Y_EngOnly_test, decision_pred_EngOnly)
'''

'\ndecision_pred_EngOnly = mtTreeEngOnly.predict(X_test)\n#decision_pred_EngOnly.shape\nmean_absolute_error(Y_EngOnly_test, decision_pred_EngOnly)\n'

In [51]:
Y_Categories[[ c for c in col_namesY]].max()

Knowledge of official languages (5):English only[2]                  224905
Knowledge of official languages (5):French only[3]                    97265
Knowledge of official languages (5):English and French[4]            133175
Knowledge of official languages (5):Neither English nor French[5]      9625
dtype: int64

## T-tests
The code below examines the differences between the 5-fold and 10-fold validation methods for each of the three models. 

Decision Trees:

In [59]:
ttestEngMatt = ttest_ind(crossValEngOnly5['test_matthews_corrcoef'], crossValEngOnly['test_matthews_corrcoef'])
ttestEngAbs = ttest_ind(crossValEngOnly5['test_neg_mean_absolute_error'], crossValEngOnly['test_neg_mean_absolute_error'])
ttestEngSquare = ttest_ind(crossValEngOnly5['test_neg_mean_squared_error'],crossValEngOnly['test_neg_mean_squared_error'])

print("English Only\n",ttestEngMatt, '\n',ttestEngAbs,'\n',ttestEngSquare, '\n')

ttestFrMatt = ttest_ind(crossValFrOnly5['test_matthews_corrcoef'], crossValFrOnly['test_matthews_corrcoef'])
ttestFrAbs = ttest_ind(crossValFrOnly5['test_neg_mean_absolute_error'], crossValFrOnly['test_neg_mean_absolute_error'])
ttestFrSquare = ttest_ind(crossValFrOnly5['test_neg_mean_squared_error'],crossValFrOnly['test_neg_mean_squared_error'])

print("French Only\n",ttestFrMatt, '\n',ttestFrAbs,'\n',ttestFrSquare)

ttestBiMatt = ttest_ind(crossValBilang5['test_matthews_corrcoef'], crossValBilang['test_matthews_corrcoef'])
ttestBiAbs = ttest_ind(crossValBilang5['test_neg_mean_absolute_error'], crossValBilang['test_neg_mean_absolute_error'])
ttestBiSquare = ttest_ind(crossValBilang5['test_neg_mean_squared_error'],crossValBilang['test_neg_mean_squared_error'])

print("Bilingual\n",ttestBiMatt, '\n',ttestBiAbs,'\n',ttestBiSquare)

ttestNeiMatt = ttest_ind(crossValNeither5['test_matthews_corrcoef'], crossValNeither['test_matthews_corrcoef'])
ttestNeiAbs = ttest_ind(crossValNeither5['test_neg_mean_absolute_error'], crossValNeither['test_neg_mean_absolute_error'])
ttestNeiSquare = ttest_ind(crossValNeither5['test_neg_mean_squared_error'],crossValNeither['test_neg_mean_squared_error'])

print("Neither\n",ttestNeiMatt, '\n',ttestNeiAbs,'\n',ttestNeiSquare)

English Only
 Ttest_indResult(statistic=-0.6738593091589552, pvalue=0.5122048750113432) 
 Ttest_indResult(statistic=-0.07122906186139671, pvalue=0.9442995952164371) 
 Ttest_indResult(statistic=0.0010020552973919227, pvalue=0.9992156894920718) 

French Only
 Ttest_indResult(statistic=-0.9800043403559678, pvalue=0.34497071461527806) 
 Ttest_indResult(statistic=-0.07329698525616048, pvalue=0.9426855716329506) 
 Ttest_indResult(statistic=0.00011447472441263734, pvalue=0.9999104004085874)
Bilingual
 Ttest_indResult(statistic=0.21703612906027042, pvalue=0.831549238466061) 
 Ttest_indResult(statistic=-0.07981476860303947, pvalue=0.9376001697742756) 
 Ttest_indResult(statistic=-0.006784893895899928, pvalue=0.9946894941011726)
Neither
 Ttest_indResult(statistic=-0.037454306356804815, pvalue=0.970691820282645) 
 Ttest_indResult(statistic=-0.00818195116688814, pvalue=0.9935960477001329) 
 Ttest_indResult(statistic=-0.0006929952218329842, pvalue=0.9994575913255996)


As the pvalue for all of these is above 0.01, we do not reject the null hypothesis that these come from the same population. I.e. there is no significant difference between the 5-fold cross-validation and the 10-fold cross-validation for the decision tree model. 

Linear Regression: 

In [61]:
#ttestEngMattLR = ttest_ind(engOnlyLR['test_matthews_corrcoef'], engOnlyLR['test_matthews_corrcoef'])
ttestEngAbsLR = ttest_ind(engOnlyLR['test_neg_mean_absolute_error'], engOnlyLR10['test_neg_mean_absolute_error'])
ttestEngSquareLR = ttest_ind(engOnlyLR['test_neg_mean_squared_error'],engOnlyLR10['test_neg_mean_squared_error'])

print("English Only\n",ttestEngAbsLR,'\n',ttestEngSquareLR, '\n')

#ttestFrMattLR = ttest_ind(frOnlyLR['test_matthews_corrcoef'], frOnlyLR['test_matthews_corrcoef'])
ttestFrAbsLR = ttest_ind(frOnlyLR['test_neg_mean_absolute_error'], frOnlyLR10['test_neg_mean_absolute_error'])
ttestFrSquareLR = ttest_ind(frOnlyLR['test_neg_mean_squared_error'],frOnlyLR10['test_neg_mean_squared_error'])

print("French Only\n",'\n',ttestFrAbsLR,'\n',ttestFrSquareLR)

#ttestBiMatt = ttest_ind(crossValBilang5['test_matthews_corrcoef'], crossValBilang['test_matthews_corrcoef'])
ttestBiAbsLR = ttest_ind(bilangLR['test_neg_mean_absolute_error'], bilangLR10['test_neg_mean_absolute_error'])
ttestBiSquareLR = ttest_ind(bilangLR['test_neg_mean_squared_error'],bilangLR10['test_neg_mean_squared_error'])

print("Bilingual\n", '\n',ttestBiAbsLR,'\n',ttestBiSquareLR)

#ttestNeiMatt = ttest_ind(crossValNeither5['test_matthews_corrcoef'], crossValNeither['test_matthews_corrcoef'])
ttestNeiAbs = ttest_ind(neitherLR['test_neg_mean_absolute_error'], neitherLR10['test_neg_mean_absolute_error'])
ttestNeiSquare = ttest_ind(neitherLR['test_neg_mean_squared_error'],neitherLR10['test_neg_mean_squared_error'])

print("Neither\n", '\n',ttestNeiAbs,'\n',ttestNeiSquare)

English Only
 Ttest_indResult(statistic=-0.001280578931511478, pvalue=0.998997688650298) 
 Ttest_indResult(statistic=2.3110141765807803e-05, pvalue=0.999981911646654) 

French Only
 
 Ttest_indResult(statistic=-0.014106008310877664, pvalue=0.9889595938907919) 
 Ttest_indResult(statistic=-1.8354054593736034e-05, pvalue=0.9999856342454234)
Bilingual
 
 Ttest_indResult(statistic=-0.0024469353410866103, pvalue=0.9980847809610536) 
 Ttest_indResult(statistic=-2.390616605565168e-05, pvalue=0.9999812885968791)
Neither
 
 Ttest_indResult(statistic=-0.0025541152530156107, pvalue=0.9980008913159713) 
 Ttest_indResult(statistic=2.7015027866552288e-05, pvalue=0.9999788552846347)


As the pvalue for all of these is above 0.01, we do not reject the null hypothesis that these come from the same population. I.e. there is no significant difference between the 5-fold cross-validation and the 10-fold cross-validation for the linear regression model. 

Random Forest: 

In [62]:
#ttestEngMattLR = ttest_ind(engOnlyLR['test_matthews_corrcoef'], engOnlyLR['test_matthews_corrcoef'])
ttestEngAbsRF = ttest_ind(engOnlyRF5['test_neg_mean_absolute_error'], engOnlyRF10['test_neg_mean_absolute_error'])
ttestEngSquareRF = ttest_ind(engOnlyRF5['test_neg_mean_squared_error'],engOnlyRF10['test_neg_mean_squared_error'])

print("English Only\n",ttestEngAbsRF,'\n',ttestEngSquareRF, '\n')

#ttestFrMattLR = ttest_ind(frOnlyLR['test_matthews_corrcoef'], frOnlyLR['test_matthews_corrcoef'])
ttestFrAbsRF = ttest_ind(frOnlyRF5['test_neg_mean_absolute_error'], frOnlyRF10['test_neg_mean_absolute_error'])
ttestFrSquareRF = ttest_ind(frOnlyRF5['test_neg_mean_squared_error'],frOnlyRF10['test_neg_mean_squared_error'])

print("French Only\n",'\n',ttestFrAbsRF,'\n',ttestFrSquareRF)

#ttestBiMatt = ttest_ind(crossValBilang5['test_matthews_corrcoef'], crossValBilang['test_matthews_corrcoef'])
ttestBiAbsRF = ttest_ind(bilangRF5['test_neg_mean_absolute_error'], bilangRF10['test_neg_mean_absolute_error'])
ttestBiSquareRF = ttest_ind(bilangRF5['test_neg_mean_squared_error'],bilangRF10['test_neg_mean_squared_error'])

print("Bilingual\n", '\n',ttestBiAbsLR,'\n',ttestBiSquareLR)

#ttestNeiMatt = ttest_ind(crossValNeither5['test_matthews_corrcoef'], crossValNeither['test_matthews_corrcoef'])
ttestNeiAbsRF = ttest_ind(neitherRF5['test_neg_mean_absolute_error'], neitherRF10['test_neg_mean_absolute_error'])
ttestNeiSquareRF = ttest_ind(neitherRF5['test_neg_mean_squared_error'],neitherRF10['test_neg_mean_squared_error'])

print("Neither\n", '\n',ttestNeiAbsRF,'\n',ttestNeiSquareRF)

English Only
 Ttest_indResult(statistic=-2.255254729519548, pvalue=0.04199787187789966) 
 Ttest_indResult(statistic=-0.2445131790563954, pvalue=0.8106503065967875) 

French Only
 
 Ttest_indResult(statistic=-0.2712791272562318, pvalue=0.7904362987992007) 
 Ttest_indResult(statistic=-0.0730598384724327, pvalue=0.9428706525442314)
Bilingual
 
 Ttest_indResult(statistic=-0.0024469353410866103, pvalue=0.9980847809610536) 
 Ttest_indResult(statistic=-2.390616605565168e-05, pvalue=0.9999812885968791)
Neither
 
 Ttest_indResult(statistic=-1.302604604726378, pvalue=0.21531357686021543) 
 Ttest_indResult(statistic=-0.6657261652682064, pvalue=0.5172249844503619)


As the pvalue for all of these is above 0.01, we do not reject the null hypothesis that these come from the same population. I.e. there is no significant difference between the 5-fold cross-validation and the 10-fold cross-validation for the Random Forest model. Note that for the Mean Absolute Error for English Only is below the 0.05 confidence level. 

In [56]:
EngOnly = DecisionTreeRegressor(max_depth=8).fit(X_train,Y_EngOnly_train)
FrOnly = DecisionTreeRegressor(max_depth=8).fit(X_train,Y_FrOnly_train)
Bilang = DecisionTreeRegressor(max_depth=8).fit(X_train,Y_Bilang_train)
Neither = DecisionTreeRegressor(max_depth=8).fit(X_train,Y_Neither_train)

In [57]:
prEngOnly = EngOnly.predict(X_test)
prFrOnly = FrOnly.predict(X_test)
prBilang = Bilang.predict(X_test)
prNeither = Neither.predict(X_test)

In [58]:
print("Mean Absolute Error for: \nEnglish Only %0.2f \nFrench Only %0.2f \nBilingual %0.2f \nNeither %0.2f" %(mean_absolute_error(Y_EngOnly_test, prEngOnly),mean_absolute_error(Y_FrOnly_test, prFrOnly),mean_absolute_error(Y_Bilang_test, prBilang), mean_absolute_error(Y_Neither_test, prNeither)))
print('\n\n')
print("Mean Squared Error for: \nEnglish Only %0.2f \nFrench Only %0.2f \nBilingual %0.2f \nNeither %0.2f" %(mean_squared_error(Y_EngOnly_test, prEngOnly),mean_squared_error(Y_FrOnly_test, prFrOnly),mean_squared_error(Y_Bilang_test, prBilang), mean_squared_error(Y_Neither_test, prNeither)))

Mean Absolute Error for: 
English Only 15.31 
French Only 1.59 
Bilingual 3.24 
Neither 0.84



Mean Squared Error for: 
English Only 67883.19 
French Only 10583.69 
Bilingual 5556.19 
Neither 242.97


In [59]:
'''
predictions = [prEngOnly, prFrOnly, prBilang, prNeither]
actual = [Y_EngOnly_test, Y_FrOnly_test, Y_Bilang_test, Y_Neither_test]
for p in predictions: 
    print(p)
    '''

'\npredictions = [prEngOnly, prFrOnly, prBilang, prNeither]\nactual = [Y_EngOnly_test, Y_FrOnly_test, Y_Bilang_test, Y_Neither_test]\nfor p in predictions: \n    print(p)\n    '

In [60]:
'''
model = MultiOutputClassifier(LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42))
model.fit(X_train,Y_train)
'''

"\nmodel = MultiOutputClassifier(LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42))\nmodel.fit(X_train,Y_train)\n"

In [61]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y_Categories, test_size=0.25,random_state=1234)
print(len(X_train),len(Y_train),len(X_test),len(Y_test))

683892 683892 227964 227964


In [62]:
Y_EngOnly_train = Y_train['Knowledge of official languages (5):English only[2]']
Y_EngOnly_test = Y_test['Knowledge of official languages (5):English only[2]']
Y_FrOnly_train = Y_train['Knowledge of official languages (5):French only[3]']
Y_FrOnly_test = Y_test['Knowledge of official languages (5):French only[3]']
Y_Bilang_train = Y_train['Knowledge of official languages (5):English and French[4]']
Y_Bilang_test = Y_test['Knowledge of official languages (5):English and French[4]']
Y_Neither_train = Y_train['Knowledge of official languages (5):Neither English nor French[5]']
Y_Neither_test = Y_test['Knowledge of official languages (5):Neither English nor French[5]']

In [63]:
crossValEngOnly5 = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_EngOnly_train),X_train,Y_EngOnly_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))
crossValFrOnly5 = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_FrOnly_train),X_train,Y_FrOnly_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))
crossValBilang5 = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_Bilang_train),X_train,Y_Bilang_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))
crossValNeither5 = cross_validate(DecisionTreeClassifier(max_depth=8).fit(X_train,Y_Neither_train),X_train,Y_Neither_train, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error','matthews_corrcoef'))


/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/grace/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [64]:
print("5 Fold Cross Validation")
print("Metrics for English Only \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValEngOnly5['test_matthews_corrcoef'].mean(), crossValEngOnly5['test_matthews_corrcoef'].std(), crossValEngOnly5['test_neg_mean_absolute_error'].mean(),crossValEngOnly5['test_neg_mean_absolute_error'].std(), crossValEngOnly5['test_neg_mean_squared_error'].mean(), crossValEngOnly5['test_neg_mean_squared_error'].std()))

print("\nMatthews Correlation ", crossValEngOnly5['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValEngOnly5['test_neg_mean_absolute_error'] )
print("Mean Squared Error " ,crossValEngOnly5['test_neg_mean_squared_error'])
print("Fit time ", crossValEngOnly5['fit_time'].mean())
print("Score time ", crossValEngOnly5['score_time'].mean())
print("\n\n")
print("Metrics for French Only \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValFrOnly5['test_matthews_corrcoef'].mean(),crossValFrOnly5['test_matthews_corrcoef'].std(), crossValFrOnly5['test_neg_mean_absolute_error'].mean(),crossValFrOnly5['test_neg_mean_absolute_error'].std(), crossValFrOnly5['test_neg_mean_squared_error'].mean(),crossValFrOnly5['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValFrOnly5['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValFrOnly5['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValFrOnly5['test_neg_mean_squared_error'])
print("Fit time ", crossValFrOnly5['fit_time'].mean())
print("Score time ", crossValFrOnly5['score_time'].mean())
print("\n\n")
print("Metrics for Both English and French \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValBilang5['test_matthews_corrcoef'].mean(),crossValBilang5['test_matthews_corrcoef'].std(), crossValBilang5['test_neg_mean_absolute_error'].mean(),crossValBilang5['test_neg_mean_absolute_error'].std(), crossValBilang5['test_neg_mean_squared_error'].mean(), crossValBilang5['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValBilang5['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValBilang5['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValBilang5['test_neg_mean_squared_error'])
print("Fit time ", crossValBilang5['fit_time'].mean())
print("Score time ", crossValBilang5['score_time'].mean())
print("\n\n")
print("Metrics for Neither English nor French \n\n %0.2f mean and %0.2f std Matthews Correlation Coefficient \n %0.2f mean and %0.2f std Mean Absolute Error \n %0.2f mean and %0.2f std Mean Squared Error" 
      % (crossValNeither5['test_matthews_corrcoef'].mean(),crossValNeither5['test_matthews_corrcoef'].std(), crossValNeither5['test_neg_mean_absolute_error'].mean(),crossValNeither5['test_neg_mean_absolute_error'].std(), crossValNeither5['test_neg_mean_squared_error'].mean(),crossValNeither5['test_neg_mean_squared_error'].std()))
print("\nMatthews Correlation ", crossValNeither5['test_matthews_corrcoef'])
print("Mean Absolute Error ", crossValNeither5['test_neg_mean_absolute_error'] )
print("Mean Squared Error" ,crossValNeither5['test_neg_mean_squared_error'])
print("Fit time ", crossValNeither5['fit_time'].mean())
print("Score time ", crossValNeither5['score_time'].mean())

5 Fold Cross Validation
Metrics for English Only 

 0.13 mean and 0.00 std Matthews Correlation Coefficient 
 -19.64 mean and 3.16 std Mean Absolute Error 
 -645953.63 mean and 245239.90 std Mean Squared Error

Matthews Correlation  [0.13736867 0.1310792  0.13238447 0.13252682 0.12314826]
Mean Absolute Error  [-17.55485126 -18.00689433 -19.57405431 -25.76587609 -17.31864774]
Mean Squared Error  [ -523594.50189722  -446144.36042814  -603887.60400064 -1126160.28765591
  -529981.41952653]
Fit time  1.4458868026733398
Score time  0.4089373588562012



Metrics for French Only 

 0.24 mean and 0.01 std Matthews Correlation Coefficient 
 -2.86 mean and 0.55 std Mean Absolute Error 
 -93278.75 mean and 34798.58 std Mean Squared Error

Matthews Correlation  [0.26009135 0.2480817  0.23948608 0.22579342 0.23692184]
Mean Absolute Error  [-2.5287142  -3.72586435 -2.77135212 -2.12450102 -3.1435611 ]
Mean Squared Error [ -64462.6592898  -157896.18837687  -94373.62039217  -61283.29318311
  -88377.9675

In [65]:
ttestEngMatt = ttest_ind(crossValEngOnly5['test_matthews_corrcoef'], crossValEngOnly['test_matthews_corrcoef'])
ttestEngAbs = ttest_ind(crossValEngOnly5['test_neg_mean_absolute_error'], crossValEngOnly['test_neg_mean_absolute_error'])
ttestEngSquare = ttest_ind(crossValEngOnly5['test_neg_mean_squared_error'],crossValEngOnly['test_neg_mean_squared_error'])
print(ttestEngMatt, '\n',ttestEngAbs,'\n',ttestEngSquare, '\n\n')


ttestFrMatt = ttest_ind(crossValFrOnly5['test_matthews_corrcoef'], crossValFrOnly['test_matthews_corrcoef'])
ttestFrAbs = ttest_ind(crossValFrOnly5['test_neg_mean_absolute_error'], crossValFrOnly['test_neg_mean_absolute_error'])
ttestFrSquare = ttest_ind(crossValFrOnly5['test_neg_mean_squared_error'],crossValFrOnly['test_neg_mean_squared_error'])
print(ttestFrMatt, '\n',ttestFrAbs,'\n',ttestFrSquare, '\n\n')

ttestBiMatt = ttest_ind(crossValBilang5['test_matthews_corrcoef'], crossValBilang['test_matthews_corrcoef'])
ttestBiAbs = ttest_ind(crossValBilang5['test_neg_mean_absolute_error'], crossValBilang['test_neg_mean_absolute_error'])
ttestBiSquare = ttest_ind(crossValBilang5['test_neg_mean_squared_error'],crossValBilang['test_neg_mean_squared_error'])
print(ttestBiMatt, '\n',ttestBiAbs,'\n',ttestBiSquare,'\n\n')

ttestNeiMatt = ttest_ind(crossValNeither5['test_matthews_corrcoef'], crossValNeither['test_matthews_corrcoef'])
ttestNeiAbs = ttest_ind(crossValNeither5['test_neg_mean_absolute_error'], crossValNeither['test_neg_mean_absolute_error'])
ttestNeiSquare = ttest_ind(crossValNeither5['test_neg_mean_squared_error'],crossValNeither['test_neg_mean_squared_error'])
print(ttestNeiMatt, '\n',ttestNeiAbs,'\n',ttestNeiSquare)

Ttest_indResult(statistic=1.1059072986393774, pvalue=0.28881295287831443) 
 Ttest_indResult(statistic=0.9673462915519547, pvalue=0.35102375862848845) 
 Ttest_indResult(statistic=0.7720146266388094, pvalue=0.4539120776701847) 


Ttest_indResult(statistic=1.8363965728681428, pvalue=0.08926577947532348) 
 Ttest_indResult(statistic=0.07689430086994184, pvalue=0.9398784823012656) 
 Ttest_indResult(statistic=0.11903069588091582, pvalue=0.9070707450441144) 


Ttest_indResult(statistic=0.18890968083607496, pvalue=0.8530809112546577) 
 Ttest_indResult(statistic=0.9992280138834634, pvalue=0.33592109408675797) 
 Ttest_indResult(statistic=0.9841141801324046, pvalue=0.3430214982124853) 


Ttest_indResult(statistic=1.475249112631501, pvalue=0.1639503573377313) 
 Ttest_indResult(statistic=0.5363918774841319, pvalue=0.6007466180685779) 
 Ttest_indResult(statistic=0.36466903863993116, pvalue=0.7212219760158857)


The Matthews Correlation Coefficient (MCC) is stable under a different training set. The range for the MCC is [-1,1] and the values of 0.09 to 0.25 are all weak relationships. This is noticeably a stronger relationship for French Only and for the Bilingual outputs, but still a weakly positive relationship. 

Both the Mean Absolute Error (MAE) and the Mean Squared Error (MSE) have the range (infinity,0] in the above tests, with the higher values closer to 0 indicating better prediction as the errors are smaller closer to 0. However, while the Mean Absolute Error represents the average size of the errors without considering the direction, the Mean Squared Error takes into account which direction the errors occurred in. The MSE gives high weight to large errors. In the metrics given above, while the MAE is relatively small, the MSE is large. This indicates that there is a great variance in the errors of the test set. 



14.2 s ± 248 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

5 Fold Cross Validation
Metrics for Bilingual 

  -13.57 mean and 0.25 std Mean Absolute Error 
 -126184.80 mean and 59721.68 std Mean Squared Error
Mean Absolute Error  [-13.6471532  -13.12055962 -13.49926601 -13.73359451 -13.86214804]
Mean Squared Error  [-134544.95292742  -39379.41031673 -104655.26729942 -127349.37748274
 -224994.98204926]
Fit time  0.4804145336151123
Score time  0.09563021659851074


5 Fold Cross Validation
Metrics for Neither 

  -1.54 mean and 0.03 std Mean Absolute Error 
 -1329.27 mean and 214.96 std Mean Squared Error
Mean Absolute Error  [-1.53417813 -1.51962314 -1.58555848 -1.54811396 -1.49765737]
Mean Squared Error  [-1560.60096831 -1126.06646474 -1216.24166213 -1616.50408879
 -1126.92400202]
Fit time  0.471939754486084
Score time  0.09939084053039551


In [82]:
engOnlyRF = RandomForestRegressor()

In [85]:
engOnlyRF.fit(X_train, Y_EngOnly_train)

RandomForestRegressor()

In [87]:
prEngOnlyRF = engOnlyRF.predict(X_test)

In [91]:
print("Mean Absolute Error", mean_absolute_error(Y_EngOnly_test,prEngOnlyRF).mean(), mean_absolute_error(Y_EngOnly_test,prEngOnlyRF).std())
print("Mean Squared Error", mean_squared_error(Y_EngOnly_test,prEngOnlyRF).mean(),mean_squared_error(Y_EngOnly_test,prEngOnlyRF).std())



Mean Absolute Error 4.204252864487375 0.0
Mean Squared Error 30904.825010615703 0.0
